In [1]:
#hide
#default_exp actions
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import show_doc
from nbdev.export import notebook2script
import shutil

# Actions

> Functions and scripts that act on a KiCad project

* toc: true

In [2]:
#export
import os
import sys
import pkg_resources
import subprocess
from pprint import pprint
import datetime as dt

import jinja2
from fastcore.script import *
import pandas as pd

from kicad_helpers import *
from kicad_helpers.utilities import _set_root, _run_cmd, _print_cmd_output

In [3]:
#hide
root = os.path.join(get_git_root("."), "_temp")
setup_test_repo(root)

In [4]:
#export
def update_templates(v:Param("verbose", bool),
                     overwrite:Param("overwrite existing templates", bool),
                     root:Param("project root directory", str)="."):
    """Install templates from the `kicad_helpers/templates` directory (ignoring
    anything in the project's `.gitignore` list).
    """
    templates_path = os.path.abspath(pkg_resources.resource_filename('kicad_helpers', 'templates'))
    root = _set_root(root)
    metadata = get_project_metadata(root)
    file_list = []
    exists_flag = False
    for root_, dirs, files in os.walk(templates_path):
        if len(files):
            for file in files:
                path = os.path.join(root_[len(templates_path) + 1:], file)
                if not in_gitignore(path, root):
                    src_path = os.path.abspath(os.path.join(templates_path, path))
                    dst_path = os.path.abspath(os.path.join(root, path))
                    
                    # Create the `dst_path` directory if it doesn't exist
                    os.makedirs(os.path.split(dst_path)[0], exist_ok=True)

                    if os.path.exists(dst_path):
                        if not overwrite:
                            print(f"{ path } already exists")
                            exists_flag = True
                            continue
    
                    with open(src_path) as f:
                        template = jinja2.Template(f.read())

                    with open(dst_path, "w") as f:
                        if v:
                            print(f"Render { path } template.")
                        f.write(template.render(**metadata))
                        
    if not overwrite and exists_flag:
        print("To overwrite existing files, use the --overwrite flag.")

The following templates are stored in the [kicad_helpers/templates](https://github.com/ryanfobel/kicad-helpers/tree/main/kicad_helpers/templates) folder and are included with the python package by adding `graft kicad_helpers/templates` to the [MANIFEST.in](https://github.com/ryanfobel/kicad-helpers/blob/main/MANIFEST.in) file:


In [5]:
#hide_input
cmd = (f"tree -a -I \"{ '|'.join(get_gitignore_list(root)) }\" "
    f"{ os.path.join(get_git_root('.'), 'kicad_helpers', 'templates') }"
)
output = _run_cmd(cmd)
print('\n'.join(output.splitlines()[1:-2]))

├── .github
│   └── workflows
│       └── build.yml
├── .kicad_helpers_config
│   ├── drc.yaml
│   ├── erc.yaml
│   ├── manufacturers
│   │   ├── PCBWay.yaml
│   │   └── default.yaml
│   ├── pcb_pdf.yaml
│   ├── pcb_svg.yaml
│   ├── sch_pdf.yaml
│   └── sch_svg.yaml
├── kitspace.yaml
├── settings.ini
└── tests
    └── Tests.ipynb


In [6]:
#export
def _add_lines_to_file(root, file_name, lines, v=False, prepend=False):
    file_path = os.path.join(root, file_name)
    if os.path.exists(file_path):
        with open(file_path, "r") as f:
            file_lines = f.readlines()
    else:
        file_lines = []

    for line in lines:
        if line not in file_lines:
            if v:
                print(f"Append \"{ line.strip() }\" to { file_path }")
            if prepend:
                file_lines.insert(0, line)
            else:
                file_lines.append(line)
        elif v:
            print(f"\"{ line.strip() }\" already exists in { file_path }")
    
    with open(file_path, "w") as f:
        f.writelines(file_lines)

In [7]:
#export
@call_parse
def add_badges(root:Param("project root directory", str)=".",
               v:Param("verbose", bool)=False,
               github:Param("github", bool)=False,
               kitspace:Param("kitspace", bool)=False):
    """Add badges to the README.md file."""
    root = _set_root(root)
    badges = []
    if kitspace:
        badges.append(kitspace_badge(root))
    if github:
        badges.append(github_badge(root))
    _add_lines_to_file(root,
                       "README.md",
                       [line + "\n" for line in badges],
                       v,
                       prepend=True
    )

In [8]:
#hide

# Remove badges from the README
readme_path = os.path.join(root, "README.md")
with open(readme_path, "r") as f:
    readme_lines = f.readlines()
readme_lines.remove(github_badge(root) + "\n")
readme_lines.remove(kitspace_badge(root) + "\n")
with open(readme_path, "w") as f:
    f.writelines(readme_lines)

# Confirm that the badges were removed
with open(readme_path, "r") as f:
    readme_lines = f.readlines()
assert github_badge(root) + "\n" not in readme_lines
assert kitspace_badge(root) + "\n" not in readme_lines
    
# Add the badges back
_print_cmd_output(f"kh_add_badges --github --kitspace --root { root }")

# Confirm that the badges were added successfully
with open(readme_path, "r") as f:
    readme_lines = f.readlines()
assert github_badge(root) + "\n" in readme_lines
assert kitspace_badge(root) + "\n" in readme_lines

This function is available as a command line script:

```sh
> kh_add_badges --help
```

In [9]:
#hide_input
_print_cmd_output("kh_add_badges --help")

usage: kh_add_badges [-h] [--root ROOT] [--v] [--github] [--kitspace]

Add badges to the README.md file.

optional arguments:
  -h, --help   show this help message and exit
  --root ROOT  project root directory (default: .)
  --v          verbose (default: False)
  --github     github (default: False)
  --kitspace   kitspace (default: False)



In [10]:
#export
_gitattributes_list = ["*.pro filter=kicad_project",
    "*.sch filter=kicad_sch"
]

def install_git_filters(root=".", v=False):
    """Install git filters to prevent insignificant changes to the kicad
    `*.pro` and `*.sch` files from being tracked by git.

    See: https://jnavila.github.io/plotkicadsch/
    """
    root = _set_root(root)

    # Add filters for kicad project and schematic files
    _add_lines_to_file(root,
                       ".gitattributes",
                       [line + "\n" for line in _gitattributes_list],
                       v
    )
    if v:
        print("Add filters to git config.")
        
    # Add filters to the project's git config
    _run_cmd(f"cd { root } && git config filter.kicad_project.clean \"sed -E 's/^update=.*$/update=Date/'\"")
    _run_cmd(f"cd { root } && git config filter.kicad_project.smudge cat")
    
    # This seems to break ERC checking. Disable for now...
    #_run_cmd(f"cd { root } && git config filter.kicad_sch.clean \"sed -E 's/#(PWR|FLG)[0-9]+/#\1?/'\"")
    #_run_cmd(f"cd { root } && git config filter.kicad_sch.smudge cat")

In [11]:
#hide

# Test adding the filters to the .gitattributes file
gitattr_path = os.path.join(root, ".gitattributes")
os.remove(gitattr_path)
install_git_filters(root)
if os.path.exists(gitattr_path):
    with open(gitattr_path, "r") as f:
        gitattr = f.readlines()
for line in _gitattributes_list:
    assert line + "\n" in gitattr  

# Test that the filters have been added to git config
filters = _run_cmd(f"cd { root } && git config --list").splitlines()
assert "filter.kicad_project.clean=sed -E 's/^update=.*$/update=Date/'" in filters
assert "filter.kicad_project.smudge=cat" in filters
# This seems to break ERC checking. Disable for now...
#assert "filter.kicad_sch.clean=sed -E 's/#(PWR|FLG)[0-9]+/#\x01?/'" in filters
#assert "filter.kicad_sch.smudge=cat" in filters

In [12]:
#export

_gitignore_list = ["_autosave*",
    "*bak",
    "*.xml",
    ".ipynb_checkpoints",
    "*-erc.txt",
    "*-drc.txt",
    "kibot_errors.filter"
]

def update_gitignore(root=".", v=False):
    """Add the following entries to the `.gitignore` file:
    ```
        _autosave*
        *bak
        *.xml
        .ipynb_checkpoints
        *-erc.txt
        *-drc.txt
        kibot_errors.filter
    ```
    """
    # Add filters for kicad project and schematic files
    _add_lines_to_file(root,
                       ".gitignore",
                       [line + "\n" for line in _gitignore_list],
                       v
    )

In [13]:
#hide

# Test adding the .gitignore entries
gitignore_path = os.path.join(root, ".gitignore")
os.remove(gitignore_path)
update_gitignore(root)
if os.path.exists(gitignore_path):
    with open(gitignore_path, "r") as f:
        gitignore = f.readlines()

for line in _gitignore_list:
    assert line + "\n" in gitignore

In [14]:
#export
@call_parse
def update_project(v:Param("verbose", bool),
                   overwrite:Param("overwrite existing templates", bool),
                   root:Param("project root directory", str)="."):
    """Setup a new project (or update an existing project) with templates from
    the `kicad_helpers/templates` directory.
    Also installs git filters to prevent insignificant changes to the kicad
    `*.pro` and `*.sch` files from being tracked by git (see
    https://jnavila.github.io/plotkicadsch/ for more details).
    """
    update_templates(v=v, overwrite=overwrite, root=root)
    install_git_filters(root=root, v=v)
    update_gitignore(root=root, v=v)

This function is available as a command line script:

```sh
> kh_update --help
```

In [15]:
#hide_input
_print_cmd_output("kh_update --help")

usage: kh_update [-h] [--v] [--overwrite] [--root ROOT]

Setup a new project (or update an existing project) with templates from the `kicad_helpers/templates` directory. Also
installs git filters to prevent insignificant changes to the kicad `*.pro` and `*.sch` files from being tracked by git
(see https://jnavila.github.io/plotkicadsch/ for more details).

optional arguments:
  -h, --help   show this help message and exit
  --v          verbose (default: False)
  --overwrite  overwrite existing templates (default: False)
  --root ROOT  project root directory (default: .)



```sh
> kh_update --v --overwrite
```

In [16]:
#hide_input
_print_cmd_output(f"kh_update --v --overwrite --root { root }")

Render kitspace.yaml template.
Render settings.ini template.
Render tests/Tests.ipynb template.
Render .github/workflows/build.yml template.
Render .kicad_helpers_config/sch_pdf.yaml template.
Render .kicad_helpers_config/pcb_svg.yaml template.
Render .kicad_helpers_config/erc.yaml template.
Render .kicad_helpers_config/sch_svg.yaml template.
Render .kicad_helpers_config/pcb_pdf.yaml template.
Render .kicad_helpers_config/drc.yaml template.
Render .kicad_helpers_config/manufacturers/default.yaml template.
Render .kicad_helpers_config/manufacturers/PCBWay.yaml template.
"*.pro filter=kicad_project" already exists in /home/ryan/dev/python/kicad-helpers/_temp/.gitattributes
"*.sch filter=kicad_sch" already exists in /home/ryan/dev/python/kicad-helpers/_temp/.gitattributes
Add filters to git config.
"_autosave*" already exists in /home/ryan/dev/python/kicad-helpers/_temp/.gitignore
"*bak" already exists in /home/ryan/dev/python/kicad-helpers/_temp/.gitignore
"*.xml" already exists in /home

In [17]:
#export
@call_parse
def sch_to_bom(root:Param("project root directory", str)=".",
               v:Param("verbose", bool)=False,
               overwrite:Param("update existing schematic", bool)=False):
    """Update/create BOM from KiCad schematic.
    """
    root = _set_root(root)
    cmd = f"{ sys.executable } -m kifield -r --nobackup --overwrite --group -aq -x { get_schematic_path(root) } -i { get_bom_path(root) }"
    if v:
        print(cmd)
    _print_cmd_output(cmd)

This function can also be called via a command line script:

```sh
> kh_sch_to_bom --help
```

In [18]:
#hide_input
_print_cmd_output("kh_sch_to_bom --help")

usage: kh_sch_to_bom [-h] [--root ROOT] [--v] [--overwrite]

Update/create BOM from KiCad schematic.

optional arguments:
  -h, --help   show this help message and exit
  --root ROOT  project root directory (default: .)
  --v          verbose (default: False)
  --overwrite  update existing schematic (default: False)



Running `kh_sch_to_bom` within your project dictory will generate a BOM file (e.g., `manufacturing/default/project-name-BOM.csv`) by extracting fields from the KiCad schematic file (e.g., `project-name.sch`).

```sh
> kh_sch_to_bom --v
```

In [19]:
#hide_input
_print_cmd_output(f"kh_sch_to_bom --v --root { root }")

/home/ryan/miniconda3/envs/kh/bin/python3.9 -m kifield -r --nobackup --overwrite --group -aq -x /home/ryan/dev/python/kicad-helpers/_temp/40-channel-hv-switching-board.sch -i /home/ryan/dev/python/kicad-helpers/_temp/manufacturing/default/40-channel-hv-switching-board-BOM.csv




Here's an example of a BOM extracted from a KiCad schematic.

In [20]:
#hide_input
df = pd.read_csv(get_bom_path(root))
df

,Refs,Quantity,MPN,Manufacturer,datasheet,footprint,value
0,"C1, C3, C5, C6, C8-C13, C16",11,CL21B104KBCNNNC,Samsung,NaN,Capacitors_SMD:C_0805,0.1uF
1,"C14, C15",2,CL21C151JBANNNC,Samsung,NaN,Capacitors_SMD:C_0805,150pF
2,"C2, C4",2,T491D336K020AT,KEMET,NaN,Sci-Bots:SM2917,33uF
3,C7,1,CL21B103KCANNNC,Samsung,NaN,Capacitors_SMD:C_0805,0.01uF
4,CH0-CH39,40,AQW214EAZ,Panasonic,NaN,SMD_Packages:DIP-8_SMD,AQW214
5,DS1,1,150080BS75000,Würth Elektronik,NaN,LEDs:LED_0805,+3.3V_PWR
6,FB1-FB5,5,742792040,Würth Elektronik,NaN,Resistors_SMD:R_0805,FERRITE
7,JP1,1,DNP,DNP,NaN,Resistors_SMD:R_0805,JUMPER
8,P1,1,DNP,DNP,NaN,Pin_Headers:Pin_Header_Angled_1x06,CONN_01X06
9,P2,1,DNP,DNP,NaN,Pin_Headers:Pin_Header_Straight_2x03,CONN_01X06


In [21]:
#export
@call_parse
def bom_to_sch(root:Param("project root directory", str)=".",
               v:Param("verbose", bool)=False,
               overwrite:Param("update existing schematic", bool)=False):
    """Update KiCad schematic from BOM file.
    """
    root = _set_root(root)
    cmd = f"{ sys.executable } -m kifield -r --nobackup --overwrite --fields ~quantity -x { get_bom_path(root) } -i { get_schematic_path(root) }"
    if v:
        print(cmd)
    _print_cmd_output(cmd)

This function can also be called via a command line script:

```sh
> kh_bom_to_sch --help
```

In [22]:
#hide_input
_print_cmd_output("kh_bom_to_sch --help")

usage: kh_bom_to_sch [-h] [--root ROOT] [--v] [--overwrite]

Update KiCad schematic from BOM file.

optional arguments:
  -h, --help   show this help message and exit
  --root ROOT  project root directory (default: .)
  --v          verbose (default: False)
  --overwrite  update existing schematic (default: False)



Running `kh_bom_to_sch` within your project dictory will read the BOM file (e.g., `manufacturing/default/project-name-BOM.csv`), and import the fields back into the KiCad schematic file (e.g., `project-name.sch`).

```sh
> kh_sch_to_bom --v
```

In [23]:
#hide_input
_print_cmd_output(f"kh_bom_to_sch --v --root { root }")

/home/ryan/miniconda3/envs/kh/bin/python3.9 -m kifield -r --nobackup --overwrite --fields ~quantity -x /home/ryan/dev/python/kicad-helpers/_temp/manufacturing/default/40-channel-hv-switching-board-BOM.csv -i /home/ryan/dev/python/kicad-helpers/_temp/40-channel-hv-switching-board.sch




In [24]:
#export
@call_parse
def export_manufacturing(root:Param("project root directory", str)=".",
                         manufacturer:Param(f"\"default\" or manufacturer name", str)="default",
                         v:Param("verbose", bool)=False,
                         output:Param("output path relative to ROOT")="."):
    """Export manufacturing files (gerber, drill, and position) by running
    KiBot in a local docker container.
    """
    root = _set_root(root)
    if manufacturer not in get_manufacturers(root):
        raise RuntimeError(f"MANUFACTURER must be one of the following: { ', '.join(get_manufacturers(root)) }.")
    
    config = f".kicad_helpers_config/manufacturers/{ manufacturer }.yaml"
    run_kibot_docker(config=config, root=root, v=v, output=output)

This function can also be called via a command line script:

```sh
> kh_export_man --help
```

In [25]:
#hide_input
_print_cmd_output("kh_export_man --help")

usage: kh_export_man [-h] [--root ROOT] [--manufacturer MANUFACTURER] [--v] [--output OUTPUT]

Export manufacturing files (gerber, drill, and position) by running KiBot in a local docker container.

optional arguments:
  -h, --help                   show this help message and exit
  --root ROOT                  project root directory (default: .)
  --manufacturer MANUFACTURER  "default" or manufacturer name (default: default)
  --v                          verbose (default: False)
  --output OUTPUT              output path relative to ROOT (default: .)



Running `kh_export_man` within your project directory will export manufacturing outputs (e.g., gerber, drill, and position files).

```sh
> kh_export_man --output outputs
> tree outputs
```

In [26]:
#hide_input
output_path = os.path.join(root, "outputs")

def remove_test_outputs():
    """Remove test outputs if the directory already exists."""
    if os.path.exists(output_path):
        shutil.rmtree(output_path)

_print_cmd_output(f"kh_export_man --root { root } --output { output_path[len(root) + 1:] }")
outputs = _run_cmd(f"cd { os.path.join(root, output_path[len(root) + 1:]) } && tree .")
print(outputs)
assert outputs == '.\n├── gerbers\n│\xa0\xa0 ├── 40-channel-hv-switching-board-NPTH.drl\n│\xa0\xa0 ├── 40-channel-hv-switching-board.drl\n│\xa0\xa0 ├── 40-channel-hv-switching-board.gbl\n│\xa0\xa0 ├── 40-channel-hv-switching-board.gbo\n│\xa0\xa0 ├── 40-channel-hv-switching-board.gbp\n│\xa0\xa0 ├── 40-channel-hv-switching-board.gbs\n│\xa0\xa0 ├── 40-channel-hv-switching-board.gl2\n│\xa0\xa0 ├── 40-channel-hv-switching-board.gl3\n│\xa0\xa0 ├── 40-channel-hv-switching-board.gm1\n│\xa0\xa0 ├── 40-channel-hv-switching-board.gtl\n│\xa0\xa0 ├── 40-channel-hv-switching-board.gto\n│\xa0\xa0 ├── 40-channel-hv-switching-board.gtp\n│\xa0\xa0 └── 40-channel-hv-switching-board.gts\n└── position\n    ├── bottom_pos.pos\n    └── top_pos.pos\n\n2 directories, 15 files\n'
remove_test_outputs()


.
├── gerbers
│   ├── 40-channel-hv-switching-board-NPTH.drl
│   ├── 40-channel-hv-switching-board.drl
│   ├── 40-channel-hv-switching-board.gbl
│   ├── 40-channel-hv-switching-board.gbo
│   ├── 40-channel-hv-switching-board.gbp
│   ├── 40-channel-hv-switching-board.gbs
│   ├── 40-channel-hv-switching-board.gl2
│   ├── 40-channel-hv-switching-board.gl3
│   ├── 40-channel-hv-switching-board.gm1
│   ├── 40-channel-hv-switching-board.gtl
│   ├── 40-channel-hv-switching-board.gto
│   ├── 40-channel-hv-switching-board.gtp
│   └── 40-channel-hv-switching-board.gts
└── position
    ├── bottom_pos.pos
    └── top_pos.pos

2 directories, 15 files



You can generate manufacturer-specific outputs using the `--manufacturer` flag, e.g.:

```sh
> kh_export_man --manufacturer PCBWay --output outputs
```

Support for additional manufacturers can be added by creating a `*.yaml` file in the [.kicad_helpers_config/manufacturers](https://github.com/ryanfobel/kicad-helpers/tree/main/kicad_helpers/templates/.kicad_helpers_config/manufacturers) directory.

In [27]:
#hide

# Test PCBWay manufacturing outputs
_run_cmd(f"kh_export_man --v --manufacturer PCBWay --root { root } --output { output_path[len(root) + 1:] }")
outputs = _run_cmd(f"cd { os.path.join(root, output_path[len(root) + 1:]) } && tree .")
assert outputs == outputs
remove_test_outputs()

In [28]:
#hide

# Test a manufacturer that has no *.yaml configuration file
output = None
try:
    _run_cmd(f"kh_export_man --root { root } --manufacturer bad --output { output_path[len(root) + 1:] }")
except subprocess.CalledProcessError as e:
    output = e.output.decode("utf-8")
assert output.splitlines()[-1] == "RuntimeError: MANUFACTURER must be one of the following: default, PCBWay."

In [29]:
#hide 

# Test using an absolute path for OUTPUT
output = None
try:
    _run_cmd(f"kh_export_man --root { root } --output { output_path }")
except subprocess.CalledProcessError as e:
    output = e.output.decode("utf-8")
assert output.splitlines()[-1].startswith("RuntimeError: OUTPUT cannot be an absolute path; it must be relative to ROOT=")

In [30]:
#export
@call_parse
def export_sch(root:Param("project root directory", str)=".",
               ext:Param(f"svg or pdf", str)="pdf",
               v:Param("verbose", bool)=False,
               output:Param("output path relative to ROOT")="."):
    """Export the schematic by running KiBot in a local docker container.
    """
    root = _set_root(root)
    supported_types = ["svg", "pdf"]
    
    if ext not in supported_types:
        raise RuntimeError(f"EXT must be one of: { ','.join(supported_types) }.")

    config = f".kicad_helpers_config/sch_{ ext }.yaml"
    run_kibot_docker(config=config, root=root, v=v, output=output)

This function can also be called via a command line script:

```sh
> kh_export_sch --help
```

In [31]:
#hide_input
_print_cmd_output("kh_export_sch --help")

usage: kh_export_sch [-h] [--root ROOT] [--ext EXT] [--v] [--output OUTPUT]

Export the schematic by running KiBot in a local docker container.

optional arguments:
  -h, --help       show this help message and exit
  --root ROOT      project root directory (default: .)
  --ext EXT        svg or pdf (default: pdf)
  --v              verbose (default: False)
  --output OUTPUT  output path relative to ROOT (default: .)



Export the schematic(s) as a pdfs:

```sh
> kh_export_sch --ext pdf --output outputs
> tree outputs
```

In [32]:
#hide_input
_run_cmd(f"kh_export_sch --root { root } --ext pdf --output { output_path[len(root) + 1:] }")
outputs = _run_cmd(f"cd { os.path.join(root, output_path[len(root) + 1:]) } && tree .")
print(outputs)
assert outputs == '.\n└── 40-channel-hv-switching-board-schematic.pdf\n\n0 directories, 1 file\n'
remove_test_outputs()

.
└── 40-channel-hv-switching-board-schematic.pdf

0 directories, 1 file



Export the schematic(s) as svgs:

```sh
> kh_export_sch --ext svg --output outputs
> tree outputs
```

In [33]:
#hide_input
_run_cmd(f"kh_export_sch --root { root } --ext svg --output { output_path[len(root) + 1:] }")
outputs = subprocess.check_output(f"cd { os.path.join(root, output_path[len(root) + 1:]) } && tree .", shell=True).decode("utf-8")
print(outputs)
assert outputs == '.\n├── 40-channel-hv-switching-board-schematic.svg\n├── switches_0-19-switches_0-19.svg\n└── switches_20-39-switches_20-39.svg\n\n0 directories, 3 files\n'
remove_test_outputs()

.
├── 40-channel-hv-switching-board-schematic.svg
├── switches_0-19-switches_0-19.svg
└── switches_20-39-switches_20-39.svg

0 directories, 3 files



In [34]:
#export
@call_parse
def export_pcb(root:Param("project root directory", str)=".",
               ext:Param(f"svg or pdf", str)="pdf",
               v:Param("verbose", bool)=False,
               output:Param("output path relative to ROOT")="."):
    """Export the pcb layout by running KiBot in a local docker container.
    """
    root = _set_root(root)
    supported_types = ["svg", "pdf"]
    
    if ext not in supported_types:
        raise RuntimeError(f"EXT must be one of: { ','.join(supported_types) }.")
    
    config = f".kicad_helpers_config/pcb_{ ext }.yaml"
    run_kibot_docker(config=config, root=root, v=v, output=output)

This function can also be called via a command line script:

```sh
> kh_export_pcb --help
```

In [35]:
#hide_input
_print_cmd_output("kh_export_pcb --help")

usage: kh_export_pcb [-h] [--root ROOT] [--ext EXT] [--v] [--output OUTPUT]

Export the pcb layout by running KiBot in a local docker container.

optional arguments:
  -h, --help       show this help message and exit
  --root ROOT      project root directory (default: .)
  --ext EXT        svg or pdf (default: pdf)
  --v              verbose (default: False)
  --output OUTPUT  output path relative to ROOT (default: .)



Export the board layout as pdfs:

```sh
> kh_export_pcb --ext pdf --output outputs
> tree outputs
```

In [36]:
#hide_input
_print_cmd_output(f"kh_export_pcb --root { root } --ext pdf --output { output_path[len(root) + 1:] }")
outputs = _run_cmd(f"cd { os.path.join(root, output_path[len(root) + 1:]) } && tree .")
print(outputs)
assert outputs == '.\n├── 40-channel-hv-switching-board-3_3V.pdf\n├── 40-channel-hv-switching-board-B_Mask.pdf\n├── 40-channel-hv-switching-board-B_Paste.pdf\n├── 40-channel-hv-switching-board-B_SilkS.pdf\n├── 40-channel-hv-switching-board-Back.pdf\n├── 40-channel-hv-switching-board-Edge_Cuts.pdf\n├── 40-channel-hv-switching-board-F_Mask.pdf\n├── 40-channel-hv-switching-board-F_Paste.pdf\n├── 40-channel-hv-switching-board-F_SilkS.pdf\n├── 40-channel-hv-switching-board-Front.pdf\n└── 40-channel-hv-switching-board-GND.pdf\n\n0 directories, 11 files\n'
remove_test_outputs()


.
├── 40-channel-hv-switching-board-3_3V.pdf
├── 40-channel-hv-switching-board-B_Mask.pdf
├── 40-channel-hv-switching-board-B_Paste.pdf
├── 40-channel-hv-switching-board-B_SilkS.pdf
├── 40-channel-hv-switching-board-Back.pdf
├── 40-channel-hv-switching-board-Edge_Cuts.pdf
├── 40-channel-hv-switching-board-F_Mask.pdf
├── 40-channel-hv-switching-board-F_Paste.pdf
├── 40-channel-hv-switching-board-F_SilkS.pdf
├── 40-channel-hv-switching-board-Front.pdf
└── 40-channel-hv-switching-board-GND.pdf

0 directories, 11 files



Export the board layout as svgs:

```sh
> kh_export_pcb --ext svg --output outputs
> tree outputs
```

In [37]:
#hide_input
_run_cmd(f"kh_export_pcb --root { root } --ext svg --output { output_path[len(root) + 1:] }")
outputs = _run_cmd(f"cd { os.path.join(root, output_path[len(root) + 1:]) } && tree .")
print(outputs)
assert outputs == '.\n├── 40-channel-hv-switching-board-3_3V.svg\n├── 40-channel-hv-switching-board-B_Mask.svg\n├── 40-channel-hv-switching-board-B_Paste.svg\n├── 40-channel-hv-switching-board-B_SilkS.svg\n├── 40-channel-hv-switching-board-Back.svg\n├── 40-channel-hv-switching-board-Edge_Cuts.svg\n├── 40-channel-hv-switching-board-F_Mask.svg\n├── 40-channel-hv-switching-board-F_Paste.svg\n├── 40-channel-hv-switching-board-F_SilkS.svg\n├── 40-channel-hv-switching-board-Front.svg\n└── 40-channel-hv-switching-board-GND.svg\n\n0 directories, 11 files\n'
remove_test_outputs()

.
├── 40-channel-hv-switching-board-3_3V.svg
├── 40-channel-hv-switching-board-B_Mask.svg
├── 40-channel-hv-switching-board-B_Paste.svg
├── 40-channel-hv-switching-board-B_SilkS.svg
├── 40-channel-hv-switching-board-Back.svg
├── 40-channel-hv-switching-board-Edge_Cuts.svg
├── 40-channel-hv-switching-board-F_Mask.svg
├── 40-channel-hv-switching-board-F_Paste.svg
├── 40-channel-hv-switching-board-F_SilkS.svg
├── 40-channel-hv-switching-board-Front.svg
└── 40-channel-hv-switching-board-GND.svg

0 directories, 11 files



In [38]:
#export
@call_parse
def run_erc(root:Param("project root directory", str)=".",
            v:Param("verbose", bool)=False):
    """Run electrical rules check (ERC) to verify schematic connections. It
    checks for output pin conflicts, missing drivers and unconnected pins.
    Print the report to `stdout`.
    """
    root = _set_root(root)
    cmd = f"eeschema_do run_erc { get_schematic_path(root)[len(root) + 1:] } ."
    try:
        output = run_docker_cmd(cmd,
           workdir=os.path.abspath(root),
           container="setsoft/kicad_auto_test:latest",
           v=v
        )
    except subprocess.CalledProcessError as e:
        output = e.output.decode("utf-8")
    if v:
        print(output)
    erc_path = os.path.join(root, get_project_name(root) + ".erc")
    with open(erc_path, "r") as f:
        erc = f.read()
    os.remove(erc_path)
    print(erc)

This function can also be called via a command line script:

```sh
> kh_run_erc --help
```

In [39]:
#hide_input
_print_cmd_output(f"kh_run_erc --help")

usage: kh_run_erc [-h] [--root ROOT] [--v]

Run electrical rules check (ERC) to verify schematic connections. It checks for output pin conflicts, missing drivers
and unconnected pins. Print the report to `stdout`.

optional arguments:
  -h, --help   show this help message and exit
  --root ROOT  project root directory (default: .)
  --v          verbose (default: False)



The ERC report is printed to `stdout` and can be redirected to a file:

```sh
> kh_run_erc > erc_report.txt
> cat erc_report.txt
```

In [40]:
#hide_input
output = _run_cmd(f"kh_run_erc --root { root }")
print(output)
assert(output.startswith("ERC report"))

ERC report (Thu Nov 25 18:17:54 2021, Encoding UTF8 )

***** Sheet /

***** Sheet /switches_0-19/

***** Sheet /switches_20-39/

 ** ERC messages: 0  Errors 0  Warnings 0




In [41]:
#export
@call_parse
def run_drc(root:Param("project root directory", str)=".",
            v:Param("verbose", bool)=False):
    """Run design rules check (DRC) and print the report to `stdout`.
    """
    root = _set_root(root)
    cmd = f"pcbnew_do run_drc { get_board_path(root)[len(root) + 1:] } ."
    try:
        output = run_docker_cmd(cmd,
            workdir=os.path.abspath(root),
            container="setsoft/kicad_auto_test:latest",
            v=v
        )
    except subprocess.CalledProcessError as e:
        output = e.output.decode("utf-8")
    if v:
        print(output)
    drc_path = os.path.join(root, "drc_result.rpt")
    with open(drc_path, "r") as f:
        drc = f.read()
    os.remove(drc_path)
    print(drc)

This function can also be called via a command line script:

```sh
> kh_run_drc --help
```

In [42]:
#hide_input
_print_cmd_output(f"kh_run_drc --help")

usage: kh_run_drc [-h] [--root ROOT] [--v]

Run design rules check (DRC) and print the report to `stdout`.

optional arguments:
  -h, --help   show this help message and exit
  --root ROOT  project root directory (default: .)
  --v          verbose (default: False)



The DRC report is printed to `stdout` and can be redirected to a file:

```sh
> kh_run_drc > drc_report.txt
> cat drc_report.txt
```

In [43]:
#hide

# Checkout `.kicad_helpers_config/drc.yaml` because is contains filters
# overriding the default template installed via `kh_update --overwrite`
_run_cmd(f"cd { root } && git checkout .kicad_helpers_config/drc.yaml");

In [44]:
#hide_input
output = _run_cmd(f"kh_run_drc --root { root }")
print(output)
assert output.startswith("** Drc report for")

** Drc report for /workdir/40-channel-hv-switching-board.kicad_pcb **
** Created on 2021-11-25 18:18:12 **

** Found 4 DRC errors **
ErrType(45): Courtyards overlap
    @(90.400 mm, 105.600 mm): Footprint C6 on Front
    @(88.500 mm, 105.600 mm): Footprint C7 on Front
ErrType(45): Courtyards overlap
    @(37.725 mm, 111.425 mm): Footprint R99 on Front
    @(34.725 mm, 111.425 mm): Footprint R125 on Front
ErrType(45): Courtyards overlap
    @(37.750 mm, 113.475 mm): Footprint R100 on Front
    @(34.725 mm, 113.475 mm): Footprint R126 on Front
ErrType(45): Courtyards overlap
    @(44.450 mm, 110.200 mm): Footprint FB1 on Back
    @(44.450 mm, 111.900 mm): Footprint FB3 on Back

** Found 0 unconnected pads **

** End of Report **




In [45]:
#export
@call_parse
def set_date(date:Param("date (defaults to today's date)", str)=None,
             root:Param("project root directory", str)=".",
             v:Param("verbose", bool)=False):
    """Set the date in all schematic and board files.
    """
    root = _set_root(root)
    if date is None:
        date = dt.datetime.now().date().isoformat()
    update_schematic_metadata({"Date": date},
                              root=root, all_sheets=True)
    update_board_metadata({"date": date}, root=root)

The following command sets the date in all schematic and board files to today's date:

```sh
> kh_set_date
```

In [46]:
#hide_input
_run_cmd(f"kh_set_date --root { root }")
print(f"schematic_metadata = { get_schematic_metadata(root) }")
print(f"board_metadata = { get_board_metadata(root) }")

schematic_metadata = {'Title': '"40-channel HV switching board"', 'Date': '"2021-11-25"', 'Rev': '"v1.0"', 'Comp': '"Sci-Bots Inc."'}
board_metadata = {'title': '"40-channel HV switching board"', 'date': '2021-11-25', 'rev': 'v1.0', 'company': '"Sci-Bots Inc."'}


In [47]:
#export
@call_parse
def set_revision(revision:Param("revision", str),
                 root:Param("project root directory", str)=".",
                 v:Param("verbose", bool)=False):
    """Set the revision in all schematic and board files.
    """
    root = _set_root(root)
    update_schematic_metadata({"Rev": revision},
                              root=root, all_sheets=True)
    update_board_metadata({"rev": revision}, root=root)

The following command sets the revision in all schematic and board files:

```sh
> kh_set_revision v1.0
```

In [48]:
#hide_input
_run_cmd(f"kh_set_revision --root { root } v1.0")
print(f"schematic_metadata = { get_schematic_metadata(root) }")
print(f"board_metadata = { get_board_metadata(root) }")

schematic_metadata = {'Title': '"40-channel HV switching board"', 'Date': '"2021-11-25"', 'Rev': '"v1.0"', 'Comp': '"Sci-Bots Inc."'}
board_metadata = {'title': '"40-channel HV switching board"', 'date': '2021-11-25', 'rev': 'v1.0', 'company': '"Sci-Bots Inc."'}


In [49]:
#hide
notebook2script()

Converted 00_actions.ipynb.
Converted 01_test.ipynb.
Converted 02_utilities.ipynb.
Converted index.ipynb.
